#### Importação das bibliotecas necessárias

In [1]:
import pandas as pd
import json
import os
import gpxo
import gpxpy
import gpxpy.gpx
import fitdecode
import warnings
import tcxparser
from tcxreader.tcxreader import TCXReader, TCXExercise

# Suppress FutureWarning messages
warnings.simplefilter(action='ignore')

#### Extração dos dados no formato Json

In [2]:
def json_processed_data(folder_path):
    
    # Leitura de cada uma das atividades no formato json 
    activities_data = [json.load(open(os.path.join(folder_path, filename),"rb")) for filename in os.listdir(folder_path)]

    df = pd.DataFrame(activities_data)

    # Lista das metricas que serão identificadas
    metrics = ['distance', 'pace', 'speed']

    # Dicionário com as features que serão criadas com o valor de cada atividade
    df_athlete_dict = {
        'total_distance (km)': [], 
        'pace (min/km)': [],
        'speed (km/h)': []    
    }

    # Iteração no número de atividades
    for activity in range(len(df)):
        # Iteração para encontrar a linha com a métrica desejada e adicionar seu valor no dicionário com a chave certa
        for values in df['summaries'][activity]: # por atividade
            if (values['metric'] == metrics[0]): 
                df_athlete_dict['total_distance (km)'].append(round(values['value'],2))
            elif (values['metric'] == metrics[1]):
                df_athlete_dict['pace (min/km)'].append(round(values['value'],2))
            elif (values['metric'] == metrics[2]):
                df_athlete_dict['speed (km/h)'].append(round(values['value'],2))

    # Criação do DataFrame de atividades
    df_athlete_activities = pd.DataFrame(df_athlete_dict)

    # Tempo inicial da atividade
    start = pd.to_datetime(df['start_epoch_ms'],unit='ms').dt.tz_localize('UTC').dt.tz_convert('America/Sao_Paulo').dt.tz_localize(None).dt.round('s')

    # Tempo final da atividade
    end = pd.to_datetime(df['end_epoch_ms'],unit='ms').dt.tz_localize('UTC').dt.tz_convert('America/Sao_Paulo').dt.tz_localize(None).dt.round('s')

    # Criação da coluna com tempo total em minutos a partir da subtração dos tempos final e inicial
    df_athlete_activities['total_time (min)'] = (end - start).apply(lambda x: x.total_seconds() // 60)

    # Criação da coluna com tipo da atividade
    df_athlete_activities['activity_type'] = df['type']

    # Criando a coluna para a data da atividade
    activity_dates = pd.to_datetime(df['start_epoch_ms'], unit='ms').dt.tz_localize('UTC').dt.tz_convert('America/Sao_Paulo').dt.tz_localize(None).dt.normalize()
    df_athlete_activities.insert(0, 'activity_date', activity_dates)

    # Ordenando a data da menor pra maior e resetando index 
    df_athlete_activities.sort_values('activity_date', ascending=True, inplace=True)    
    df_athlete_activities.reset_index(drop=True, inplace=True)

    return df_athlete_activities

#### Extração dos dados no formato .fit

In [3]:
def fit_processed_data(folder_path):

    # Dicionário com as features que serão criadas com o valor de cada atividade
    df_athlete_dict = {
    'activity_date': [],
    'activity_type': [],                
    'total_distance (km)': [],        
    'total_time (min)': [],           
    'pace (min/km)': [],              
    'speed (km/h)': []             
    }

    # Itera por cada arquivo na pasta especificada
    for filename in os.listdir(folder_path):
        # Concatena o caminho da pasta com o nome do arquivo para obter o caminho completo
        full_path = os.path.join(folder_path, filename)

        # Abre e lê o arquivo .fit usando fitdecode
        with fitdecode.FitReader(full_path) as fit_file:
            # Itera sobre cada quadro de dados no arquivo .fit
            for frame in fit_file:
                # Verifica se o quadro é do tipo dado (DATA FRAME)
                if frame.frame_type == fitdecode.FIT_FRAME_DATA:
                    # Verifica se o quadro corresponde a uma sessão (session)
                    if frame.name == 'session':
                        # Itera sobre cada campo de dados na sessão
                        for field in frame.fields:
                            # Verifica se o campo é 'start_time' e armazena a data da atividade
                            if field.name == 'start_time':
                                df_athlete_dict['activity_date'].append(field.value)
                            
                            # Verifica se o campo é 'sport' e armazena o tipo da atividade
                            elif field.name == 'sport':
                                df_athlete_dict['activity_type'].append(field.value)

                            # Verifica se o campo é 'total_distance' e armazena a distância total (convertida para km)
                            elif field.name == 'total_distance':
                                total_dist_km = field.value / 1000
                                df_athlete_dict['total_distance (km)'].append(round(total_dist_km, 2))

                            # Verifica se o campo é 'total_timer_time' e armazena o tempo total (convertido para minutos)
                            elif field.name == 'total_timer_time':
                                total_time_min = field.value // 60
                                df_athlete_dict['total_time (min)'].append(total_time_min)
        
        # Calcula e armazena o pace (min/km) para a atividade atual
        if total_dist_km == 0:  # Se a distância for zero, o pace também é definido como zero
            df_athlete_dict['pace (min/km)'].append(0)
        else:  # Caso contrário, calcula o pace como o tempo total dividido pela distância total
            df_athlete_dict['pace (min/km)'].append(round(total_time_min / total_dist_km, 2))

        # Calcula e armazena a velocidade média (km/h) para a atividade atual
        df_athlete_dict['speed (km/h)'].append(round(total_dist_km / (total_time_min / 60), 2))
        
        # Passando o dicionário com as features criadas para um Dataframe
        df_athlete_activities = pd.DataFrame(df_athlete_dict)

        # Convertendo a coluna de data para o fuso horário local e arrendado em dias
        df_athlete_activities['activity_date'] = df_athlete_activities['activity_date'].dt.tz_convert('America/Sao_Paulo').dt.tz_localize(None).dt.normalize()

        # Ordenando a data da menor pra maior e resetando index 
        df_athlete_activities.sort_values('activity_date', ascending=True, inplace=True)    
        df_athlete_activities.reset_index(drop=True, inplace=True)

    return df_athlete_activities

#### Extração dos dados no formato .gpx

In [4]:
def gpx_processed_data(full_path):

    # Lê o arquivo no formato gpx com as bibliotecas gpxo e gpxpy
    gpx1 = gpxo.Track(full_path).data.reset_index()   
    gpx2 = gpxpy.parse(open(full_path, 'r'))

    # Data da atividade (a partir da primeira linha do df original com os pontos marcados)
    activity_date = gpx1['time'][0].normalize()

    # Criação da coluna com tipo da atividade
    activity_type = gpx2.tracks[0].type 

    # Distância total em km
    total_dist = round(gpx1['distance (km)'].max(), 2)

    # Tempo total em minutos
    total_min = (gpx1['time'].max() - gpx1['time'].min()).total_seconds() // 60

    # Ritmo geral da atividade
    if total_dist == 0:
        pace = 0
    else:
        pace = round(total_min / total_dist, 2)

    # Velocidade em km/h
    speed = round(total_dist / (total_min / 60),2)

    return activity_date, activity_type, total_dist, total_min, pace, speed

#### Extração dos dados no formato .tcx

In [5]:
def tcx_processed_data(full_path):

    try:
        
        tcx_reader = TCXReader()
        data: TCXExercise = tcx_reader.read(full_path)

        # Data da atividade (a partir da primeira linha do df original com os pontos marcados)
        activity_date = pd.Timestamp(data.start_time.date())

        # Criação da coluna com tipo da atividade
        activity_type = data.activity_type

        # Distância total em km
        total_dist = round(data.distance / 1000, 2)

        # Tempo total em minutos
        total_min = (data.end_time - data.start_time).total_seconds() // 60

        # Ritmo geral da atividade
        if total_dist == 0:
            pace = 0
        else:
            pace = round(total_min / total_dist, 2)

        # Velocidade em km/h
        speed = round(data.avg_speed,2)

    except:

        tcx_parser = tcxparser.TCXParser(full_path)

        # Data da atividade (a partir da primeira linha do df original com os pontos marcados)
        activity_date = pd.Timestamp(tcx_parser.started_at).tz_convert('America/Sao_Paulo').tz_localize(None).normalize()

        # Criação da coluna com tipo da atividade
        activity_type = tcx_parser.activity_type

        # Distância total em km
        total_dist = round(tcx_parser.distance / 1000, 2)

        # Tempo total em minutos
        try:
            total_min = tcx_parser.duration // 60

        # Exceção caso não haja no arquivo tcx "<TotalTimeSeconds>"
        except:
            total_min = (pd.Timestamp(tcx_parser.completed_at) - pd.Timestamp(tcx_parser.started_at)).total_seconds() // 60

        # Ritmo geral da atividade
        if total_dist == 0:
            
            pace = 0
            
        else:
            pace = round(total_min / total_dist, 2)

        # Velocidade em km/h
        speed = round(total_dist / (total_min/60), 2)

    return activity_date, activity_type, total_dist, total_min, pace, speed    

#### Iteração para extração dos dados com as funções para .tcx e/ou .gpx
#### Obs: Se a pasta tiver arquivos diferentes de .tcx e .gpx, o script não irá funcionar nele

In [6]:
def tcx_or_gpx_processed_data(folder_path): # -> return activity_date, activity_type, total_dist, total_min, pace, speed
    
    # Dicionário com as features que serão criadas com o valor de cada atividade
    df_athlete_dict = {
        'activity_date': [],
        'activity_type': [],
        'total_distance (km)': [], 
        'total_time (min)': [],
        'pace (min/km)': [],
        'speed (km/h)': []    
    }
        
    # Itera por cada arquivo na pasta
    for filename in os.listdir(folder_path):
        # Junta o nome do arquivo com o caminho da pasta
        full_path = os.path.join(folder_path, filename)

        if full_path.endswith('gpx'):
            activity_date, activity_type, total_dist, total_min, pace, speed = gpx_processed_data(full_path)       

        if full_path.endswith('tcx'):
            activity_date, activity_type, total_dist, total_min, pace, speed = tcx_processed_data(full_path)

        df_athlete_dict['activity_date'].append(activity_date)
        df_athlete_dict['activity_type'].append(activity_type)
        df_athlete_dict['total_distance (km)'].append(total_dist)
        df_athlete_dict['total_time (min)'].append(total_min)
        df_athlete_dict['pace (min/km)'].append(pace)
        df_athlete_dict['speed (km/h)'].append(speed)  

    # Passando o dicionário com as features criadas para um Dataframe
    df_athlete_activities = pd.DataFrame(df_athlete_dict)

    # Ordenando a data da menor pra maior e resetando index 
    df_athlete_activities.sort_values('activity_date', ascending=True, inplace=True)    
    df_athlete_activities.reset_index(drop=True, inplace=True)
    
    return df_athlete_activities                

#### Criação do Dataframe com todas as atividades, a partir das funções para cada formato

In [7]:
# Caminho da pasta com as pastas contendo as atividades de cada atleta
data_folders_path = '.../running_inactivity_prediction/athletes_activities'

# Contador de arquivos dentro da pasta
folders_count = len(os.listdir(data_folders_path))

# Criação do Dataframe que terá os valores de cada atleta por id
df_athletes_activities = pd.DataFrame()

# Número do primeiro id do atleta (Se preferir começar com 0, mudar para n=0)
n = 1

# Iteração para criar o Dataframe com os valores de cada atleta por id
for athlete_id in range(n, folders_count + n):

    # Nome do arquivo com os dados do atleta
    athlete_foldername = f'athlete{athlete_id}'

    # Junção da pasta com as pastas contendo as atividades de cada atleta e o nome da pasta com os dados do atleta por id
    data_folder_path = os.path.join(data_folders_path, athlete_foldername)

    # Pegando os formatos de cada arquivo na respectiva pasta de atividades
    file_formats = set([os.listdir(data_folder_path)[i].split('.')[1] for i in range(len(os.listdir(data_folder_path)))])

    if 'gpx' in file_formats or 'tcx' in file_formats:
        df_athlete_activity = tcx_or_gpx_processed_data(data_folder_path)

    if 'fit' in file_formats:
        df_athlete_activity = fit_processed_data(data_folder_path)

    if 'json' in file_formats:
        df_athlete_activity = json_processed_data(data_folder_path) 

    # Adição de uma coluna athlete_id para identificar os dados de cada atleta
    df_athlete_activity.insert(0, 'athlete_id', athlete_id)

    # Concatenação dos valores de cada atleta por id a um Dataframe
    df_athletes_activities = pd.concat([df_athletes_activities, df_athlete_activity])

# Resetando o index do dataframe final
df_athletes_activities.reset_index(drop=True, inplace=True)

#### Dataframe com features iniciais

In [8]:
df_athletes_activities

,athlete_id,activity_date,activity_type,total_distance (km),total_time (min),pace (min/km),speed (km/h)
0,1,2020-07-09,running,6.03,39.0,6.47,9.28
1,1,2020-07-10,cycling,25.35,140.0,5.52,10.86
2,1,2020-07-12,running,7.57,55.0,7.27,8.26
3,1,2020-07-16,running,3.61,23.0,6.37,9.42
4,1,2020-07-20,cycling,25.66,127.0,4.95,12.12
...,...,...,...,...,...,...,...
989,7,2024-06-21,running,12.10,62.0,5.12,11.71
990,7,2024-06-26,running,4.08,22.0,5.39,11.13
991,7,2024-06-28,running,2.03,10.0,4.93,12.18
992,7,2024-06-28,running,2.08,10.0,4.81,12.48


#### Seleção das atividades apenas de corrida

In [9]:
df_athletes_activities['activity_type'].value_counts()

activity_type
running           571
run               172
cycling            95
workout            54
Running            49
walking            12
swimming           10
training            8
ride                7
other               5
walk                4
hiking              3
54                  1
weighttraining      1
Biking              1
Name: count, dtype: int64

In [10]:
# Selecionando apenas dados de corrida
df_athletes_activities = df_athletes_activities[df_athletes_activities['activity_type'].isin(['running', 'run', 'Running'])]

# Dropando atividades duplicadas por atleta + data da atividade
df_athletes_activities.drop_duplicates(subset=['athlete_id', 'activity_date'], keep='first', inplace=True)

# Dropando coluna de tipo de atividade
df_athletes_activities.drop('activity_type', axis=1, inplace=True)

# Resetando o index do dataframe final
df_athletes_activities.reset_index(drop=True, inplace=True)

df_athletes_activities

,athlete_id,activity_date,total_distance (km),total_time (min),pace (min/km),speed (km/h)
0,1,2020-07-09,6.03,39.0,6.47,9.28
1,1,2020-07-12,7.57,55.0,7.27,8.26
2,1,2020-07-16,3.61,23.0,6.37,9.42
3,1,2020-07-24,5.84,41.0,7.02,8.55
4,1,2020-07-25,4.28,32.0,7.48,8.02
...,...,...,...,...,...,...
741,7,2024-06-16,10.06,45.0,4.47,13.26
742,7,2024-06-18,5.20,33.0,6.35,9.27
743,7,2024-06-21,12.10,62.0,5.12,11.71
744,7,2024-06-26,4.08,22.0,5.39,11.13


In [11]:
df_athletes_activities[['total_distance (km)','total_time (min)','pace (min/km)', 'speed (km/h)']].describe()

,total_distance (km),total_time (min),pace (min/km),speed (km/h)
count,746.000000,746.000000,746.000000,746.000000
mean,7.822547,44.387399,6.058190,10.403405
std,4.723389,21.306490,1.662228,2.320400
min,0.000000,2.000000,0.000000,0.000000
25%,5.010000,31.000000,4.990000,9.142500
50%,6.000000,39.000000,5.920000,10.105000
75%,10.010000,56.000000,6.550000,12.010000
max,42.460000,192.000000,17.740000,16.450000


#### Remoção de features com distância 0 pois atrapalham na criação das features

In [12]:
df_athletes_activities[df_athletes_activities['total_distance (km)'] == 0]

,athlete_id,activity_date,total_distance (km),total_time (min),pace (min/km),speed (km/h)
685,6,2023-08-14,0.0,25.0,0.0,0.0
697,6,2023-11-27,0.0,39.0,0.0,0.0


In [13]:
# Contando apenas atividades com distância acima de 0
df_athletes_activities = df_athletes_activities[df_athletes_activities['total_distance (km)'] != 0]

#### Criação de intervalo com frequência diária entre dias de atividade e sem atividade para cada atleta

In [14]:
df_athletes_activities_new = pd.DataFrame()

for id, df_grouped in df_athletes_activities.groupby('athlete_id'):

    df_grouped.insert(1, 'is_activity', True)

    # Data da primeira e última atividade para criar um range
    start, end = df_grouped['activity_date'].iloc[0], df_grouped['activity_date'].iloc[-1]

    # Colocando todas as datas em um df
    df_dates = pd.DataFrame({'activity_date': pd.date_range(start, end, freq='d')})

    # Right join pra poder manter os dados das atividades existentes mas adicionar linhas para os novos dias
    df_grouped = df_dates.merge(df_grouped, on='activity_date', how='left')

    # Preechendo os valores nulos como Dia sem atividade
    df_grouped['is_activity'].fillna(False, inplace=True)
    
    # Preenchendo novamente a coluna id para evitar NaN das linhas sem atividade
    df_grouped['athlete_id'] = id

    # Concatenação dos valores de cada atleta por id a um Dataframe
    df_athletes_activities_new = pd.concat([df_athletes_activities_new, df_grouped])

df_athletes_activities_new.reset_index(drop=True, inplace=True)

df_athletes_activities_new

,activity_date,athlete_id,is_activity,total_distance (km),total_time (min),pace (min/km),speed (km/h)
0,2020-07-09,1,True,6.03,39.0,6.47,9.28
1,2020-07-10,1,False,NaN,NaN,NaN,NaN
2,2020-07-11,1,False,NaN,NaN,NaN,NaN
3,2020-07-12,1,True,7.57,55.0,7.27,8.26
4,2020-07-13,1,False,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
4645,2024-06-24,7,False,NaN,NaN,NaN,NaN
4646,2024-06-25,7,False,NaN,NaN,NaN,NaN
4647,2024-06-26,7,True,4.08,22.0,5.39,11.13
4648,2024-06-27,7,False,NaN,NaN,NaN,NaN


#### Criação de novas features temporais com agregação de janela deslizante para cada atleta

In [15]:
window_size = 7

''' Feature dependentes da is_activity '''
df_athletes_activities_new['week_frequency'] = df_athletes_activities_new.groupby('athlete_id')['is_activity'].rolling(window_size, min_periods=1).sum().values
df_athletes_activities_new['week_frequency'] = df_athletes_activities_new['week_frequency'].astype(int)

''' Features dependentes de total_time (min) '''
df_athletes_activities_new['week_total_time'] = df_athletes_activities_new.groupby('athlete_id')['total_time (min)'].rolling(window_size, min_periods=1).sum().values
df_athletes_activities_new['week_max_time'] = df_athletes_activities_new.groupby('athlete_id')['total_time (min)'].rolling(window_size, min_periods=1).max().values

''' Features dependentes de total_distance (km) '''
df_athletes_activities_new['week_total_dist'] = df_athletes_activities_new.groupby('athlete_id')['total_distance (km)'].rolling(window_size, min_periods=1).sum().values
df_athletes_activities_new['week_max_dist'] = df_athletes_activities_new.groupby('athlete_id')['total_distance (km)'].rolling(window_size, min_periods=1).max().values

''' Features dependentes de pace (min/km) '''
df_athletes_activities_new['week_best_pace'] = df_athletes_activities_new.groupby('athlete_id')['pace (min/km)'].rolling(window_size, min_periods=1).min().values

''' Features dependentes de speed (km/h) '''
df_athletes_activities_new['week_best_speed'] = df_athletes_activities_new.groupby('athlete_id')['speed (km/h)'].rolling(window_size, min_periods=1).max().values

# Desconsiderando a coluna is_activity e as linhas sem atividade pois não serão mais úteis
df_athletes_activities_new = df_athletes_activities_new.loc[df_athletes_activities_new['is_activity'] == True, ~df_athletes_activities_new.columns.isin(['is_activity'])].reset_index(drop=True)

#### Criação da quantidade de dias sem atividade, inativação (target), retornos e taxa de inativação para cada atleta

In [16]:
# Feature para quantidade de dias desde a última atividade
df_athletes_activities_new['days_since_last_act'] = (df_athletes_activities_new.groupby('athlete_id')['activity_date'].diff().dt.days.fillna(0)).astype(int)

# Criação do target
df_athletes_activities_new['inactivation'] = df_athletes_activities_new.groupby('athlete_id')['days_since_last_act'].transform(lambda x: (x >= 5).shift(-1).fillna(0).astype(int))

# Criação de contagem de retorno acumulado do atleta pós inatividade
df_athletes_activities_new['return_count'] = (df_athletes_activities_new.groupby('athlete_id')['inactivation'].cumsum().shift(1).fillna(0)).astype(int)

# Criação de % acumulada de inatividade do atleta
df_athletes_activities_new['inactivation_rate'] = df_athletes_activities_new.groupby('athlete_id')['inactivation'].transform(lambda x: (x.cumsum()/len(x)).round(3).shift(1).fillna(0))

In [17]:
df_athletes_activities_new['inactivation'].value_counts()

inactivation
0    542
1    202
Name: count, dtype: int64

#### Criação para recorde pessoal (PR) de features criadas anteriormente para cada atleta

In [18]:
'''PR Features'''

df_athletes_activities_new['PR_total_dist'] = df_athletes_activities_new.groupby('athlete_id')['total_distance (km)'].cummax()
df_athletes_activities_new['PR_total_time'] = df_athletes_activities_new.groupby('athlete_id')['total_time (min)'].cummax()
df_athletes_activities_new['PR_pace'] = df_athletes_activities_new.groupby('athlete_id')['pace (min/km)'].cummin()
df_athletes_activities_new['PR_speed'] = df_athletes_activities_new.groupby('athlete_id')['speed (km/h)'].cummax()
df_athletes_activities_new['PR_days_since_last_act'] = df_athletes_activities_new.groupby('athlete_id')['days_since_last_act'].cummax()
df_athletes_activities_new['PR_week_frequency'] = df_athletes_activities_new.groupby('athlete_id')['week_frequency'].cummax()
df_athletes_activities_new['PR_week_total_time'] = df_athletes_activities_new.groupby('athlete_id')['week_total_time'].cummax()
df_athletes_activities_new['PR_week_max_time'] = df_athletes_activities_new.groupby('athlete_id')['week_max_time'].cummax()
df_athletes_activities_new['PR_week_total_dist'] = df_athletes_activities_new.groupby('athlete_id')['week_total_dist'].cummax()
df_athletes_activities_new['PR_week_max_dist'] = df_athletes_activities_new.groupby('athlete_id')['week_max_dist'].cummax()
df_athletes_activities_new['PR_week_best_pace'] = df_athletes_activities_new.groupby('athlete_id')['week_best_pace'].cummin()
df_athletes_activities_new['PR_week_best_speed'] = df_athletes_activities_new.groupby('athlete_id')['week_best_speed'].cummax()

#### Dataframe final

In [19]:
df_athletes_activities_new

,activity_date,athlete_id,total_distance (km),total_time (min),pace (min/km),speed (km/h),week_frequency,week_total_time,week_max_time,week_total_dist,...,PR_pace,PR_speed,PR_days_since_last_act,PR_week_frequency,PR_week_total_time,PR_week_max_time,PR_week_total_dist,PR_week_max_dist,PR_week_best_pace,PR_week_best_speed
0,2020-07-09,1,6.03,39.0,6.47,9.28,1,39.0,39.0,6.03,...,6.47,9.28,0,1,39.0,39.0,6.03,6.03,6.47,9.28
1,2020-07-12,1,7.57,55.0,7.27,8.26,2,94.0,55.0,13.60,...,6.47,9.28,3,2,94.0,55.0,13.60,7.57,6.47,9.28
2,2020-07-16,1,3.61,23.0,6.37,9.42,2,78.0,55.0,11.18,...,6.37,9.42,4,2,94.0,55.0,13.60,7.57,6.37,9.42
3,2020-07-24,1,5.84,41.0,7.02,8.55,1,41.0,41.0,5.84,...,6.37,9.42,8,2,94.0,55.0,13.60,7.57,6.37,9.42
4,2020-07-25,1,4.28,32.0,7.48,8.02,2,73.0,41.0,10.12,...,6.37,9.42,8,2,94.0,55.0,13.60,7.57,6.37,9.42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
739,2024-06-16,7,10.06,45.0,4.47,13.26,3,95.0,45.0,18.83,...,4.36,13.77,5,5,190.0,70.0,35.81,13.24,4.36,13.77
740,2024-06-18,7,5.20,33.0,6.35,9.27,3,108.0,45.0,20.96,...,4.36,13.77,5,5,190.0,70.0,35.81,13.24,4.36,13.77
741,2024-06-21,7,12.10,62.0,5.12,11.71,3,140.0,62.0,27.36,...,4.36,13.77,5,5,190.0,70.0,35.81,13.24,4.36,13.77
742,2024-06-26,7,4.08,22.0,5.39,11.13,2,84.0,62.0,16.18,...,4.36,13.77,5,5,190.0,70.0,35.81,13.24,4.36,13.77
